In [ ]:
from NODE_WAN_model.training import NODE_WAN_solver
import torch
import math
from NODE_WAN_model.dataset import *

# Form of our problem:
\begin{equation}
\left\{\begin{array}{ll}
u_{t}-\bigtriangleup u - u**2 = f, & \text { in } \Omega \times[0, T] \\
u(x, t)=g(x, t), & \text { on } \partial \Omega \times[0, T] \\
u(x, 0)=h(x), & \text { in } \Omega
\end{array}\right.
\end{equation}

where $x$ is a d-dimensional vector.

The functions are:

In [ ]:
# setting to cuda

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Setting Parameters

We have two dictionaries in which we can specify our desired problem setup (including number of points sampled, as shown
in the paper) and the configuration of our neural networks, called `setup` and `config` respectively.


In [ ]:
setup = {
    'dim': 5,   # int: dimension of the problem
    'N_t': 20,  # int: number of time samples
    'N_r': 400, # int: number of spatial samples in the interior
    'N_b': 400, # int: number of spatial samples on the boundary
    'T0': 0,    # float: initial time
    'T': 1      # float: final time
}

config = {
    'alpha': 1e4 * 400 * 25,    # float: the coefficient in our loss function
    'u_layers': 8,              # int: depth of the hidden field F
    'u_hidden_dim': 20,         # int: dimensionality of the initial and final layers
    'u_hidden_hidden_dim': 10,  # int: dimensionality of the hidden field F
    'v_layers': 9,              # int: the depth of the adversarial DNN
    'v_hidden_dim': 50,         # int: the dimensionality of the adversarial DNN
    'n1': 2,                    # int: sub-iterations for fitting of the guess
    'n2': 1,                    # int: sub-iterations for fitting of the test function
    'u_rate': 0.015,            # float: learning rate of the guess
    'v_rate': 0.04,             # float: learning rate of the test function
    'min_steps': 5,             # int: smallest number of time steps used in the ODE solver
    'adjoint': False,           # bool: whether to use an adjoint solver for the ODE solver
    'solver': 'midpoint'        # str: the solver to be used in the ODE solver
}

iterations = 1001

# Setting the specific problem to solve
Here we can specify the functions that we have from our problem, including the solution if known. In case there is an
unknown solution, do not provide `func_u_sol` to `NODE_WAN_solver` below or set it to `None`.

In [ ]:
def func_u_sol(X):
    sins = 1
    for i in range(setup['dim']):
        sins *= torch.sin(math.pi/2 * X[:, :, i+1] + math.pi/2 * i)
    return (2/math.pi) ** (-setup['dim']) * 2 * sins * torch.exp(-X[:, :, 0])


def func_f(X):
    sins = 1
    for i in range(setup['dim']):
        sins *= torch.sin(math.pi / 2 * X[:, :, i + 1] + math.pi / 2 * i)
    return (2/math.pi) ** (-setup['dim']) * (math.pi ** 2 - 2) * sins * torch.exp(-X[:, :, 0]) - 4 * sins ** 2 * torch.exp(-2*X[:, :, 0])


def func_g(BX):
    return func_u_sol(BX)


def func_h(X):
    sins = 1
    for i in range(setup['dim']):
        sins *= torch.sin(math.pi / 2 * X[:, i + 1] + math.pi / 2 * i)
    return (2/math.pi) ** (-setup['dim']) * 2 * sins


def func_a(X, i, j):
    if i == j:
        return torch.ones(X.shape[:-1])
    else:
        return torch.zeros(X.shape[:-1])


def func_b(X, i):
    return torch.zeros(X.shape[:-1])

The following function can take into account the function u, so they have the input `y_output_u` which will be our
guess solution.

In [ ]:
def func_c(X, y_output_u):
    return -y_output_u

We can now specify the domain by calling one of the shapes from `NODE_WAN_model.dataset` or a custom one (more on the
rules of this on the git or the `NODE_WAN_model.dataset` file itself).

In [ ]:
params = {**config, **setup, **{'iterations': int(iterations)}, **{'domain': 'Hypercube'}}

solver = NODE_WAN_solver(params, func_a, func_b, func_c, func_h, func_f, func_g, device, './', func_u_sol=func_u_sol, p=2)

In [ ]:
solver.train(report=True, report_it=100, show_plt=True)